# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [7]:
from util import util_pipeline

In [8]:
import importlib
# importlib.reload(util_elastic)

In [9]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [10]:
PATH_DOC =  '../data/juris_tcu_index/doc.csv'
PATH_QUERY =  '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [11]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 186 µs, sys: 15 µs, total: 201 µs
Wall time: 203 µs


# Data load

In [12]:
df_doc = pd.read_csv(PATH_DOC)

In [13]:
df_query = pd.read_csv(PATH_QUERY)

In [14]:
df_qrel = pd.read_csv(PATH_QREL)

In [15]:
df_doc.shape, df_query.shape, df_qrel.shape

((13255, 28), (16045, 11), (94809, 3))

In [16]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


### Quantitative calculation

Tokenization loading

In [17]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"

nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"

tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)

/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [18]:
nome_modelo_ptt5_base = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
nome_caminho_modelo_ptt5_base  = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ptt5_base
assert os.path.exists(nome_caminho_modelo_ptt5_base ), f"Path para {nome_caminho_modelo_ptt5_base } não existe!"

In [19]:
tokenizador_ptt5_base = AutoTokenizer.from_pretrained(nome_caminho_modelo_ptt5_base)

In [20]:
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


In [21]:
df_query['LEN_TEXT_CHAR'] = df_query['TEXT'].apply(len)
df_query['LEN_TEXT_CHAR_LOG'] = round(np.log(df_query['TEXT'].apply(len))).astype(int)
df_query['NUM_WORD'] = df_query['TEXT'].apply(lambda x: len(x.split()))
df_query['NUM_TOKENS_MONOT5_3B'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_query['NUM_TOKENS_MINILM'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)
df_query['NUM_TOKENS_PTT5_BASE'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_ptt5_base)


In [22]:
df_query.columns, df_qrel.columns, df_doc.columns, 

(Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
        'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM', 'NUM_TOKENS_PTT5_BASE'],
       dtype='object'),
 Index(['QUERY_ID', 'DOC_ID', 'TYPE'], dtype='object'),
 Index(['TEXT', 'ID', 'NAME', 'DATE_REFERENCE', 'CLASS', 'HAS_SCOPE_NOTE', 'HAS_DEFINITION', 'HAS_SYNONYM', 'HAS_EXAMPLE', 'HAS_ENGLISH_TRANSLATION', 'HAS_SPECIALIZATION', 'HAS_RELATED_TERM',
        'HAS_SPANISH_TRANSLATION', 'HAS_GENERALIZATION', 'TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION',
        'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION', 'COUNT_INDEX_AREA', 'COUNT_INDEX_THEME', 'COUNT_INDEX_SUBTHEME', 'COUNT_INDEX_EXTRA', 'COUNT_INDEX_TOTAL'],
       dtype='object'

In [23]:
# Verificar as colunas com valores NaN
columns_with_nan = df_query.columns[df_query.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [24]:
df_query = df_query.fillna("")

In [25]:
df_doc.head()

,TEXT,ID,NAME,DATE_REFERENCE,CLASS,HAS_SCOPE_NOTE,HAS_DEFINITION,HAS_SYNONYM,HAS_EXAMPLE,HAS_ENGLISH_TRANSLATION,HAS_SPECIALIZATION,HAS_RELATED_TERM,HAS_SPANISH_TRANSLATION,HAS_GENERALIZATION,TEXT_DEFINITION,TEXT_SYNONYM,TEXT_RELATED_TERM,TEXT_SCOPE_NOTE,TEXT_EXAMPLE,TEXT_ENGLISH_TRANSLATION,TEXT_SPANISH_TRANSLATION,TEXT_SPECIALIZATION,TEXT_GENERALIZATION,COUNT_INDEX_AREA,COUNT_INDEX_THEME,COUNT_INDEX_SUBTHEME,COUNT_INDEX_EXTRA,COUNT_INDEX_TOTAL
0,"O termo é ""Abandono de cargo"".\nAbandono de cargo tem definição: ""Configura abandono de cargo a ...",1,Abandono de cargo,20230614,Termo,1,1,1,0,1,0,1,1,0,Configura abandono de cargo a ausência intencional do servidor ao serviço por mais de trinta dia...,Abandono de função e Abandono do cargo.,"Abandono de emprego, Deserção, Falta grave, Função pública, Demissão de servidor, Inassiduidade ...",Artigo 323 Código Penal,NaN,Abandonment of office.,Abandono de puesto.,NaN,NaN,0,1,0,0,1
1,"O termo é ""Abastecimento de tropas e meios"".\nAbastecimento de tropas e meios é uma especializaç...",2,Abastecimento de tropas e meios,20230614,Termo,0,0,0,0,1,0,0,1,1,NaN,NaN,NaN,NaN,NaN,Supply of troops and means.,Abastecimiento de tropas y medios e Suministro de tropas y médios.,Abastecimento.,NaN,0,0,0,1,1
2,"O termo é ""Tropa"".\nTropa tem definição: ""Termo coletivo que designa o pessoal de uma organizaçã...",3,Tropa,20230614,Termo,0,1,0,0,1,0,1,1,0,Termo coletivo que designa o pessoal de uma organização militar.,NaN,Organização militar e Missão.,NaN,NaN,Troop.,Tropa.,NaN,NaN,0,0,0,0,0
3,"O termo é ""Abono de faltas"".\nAbono de faltas tem definição: ""É a relevação da falta, de forma q...",4,Abono de faltas,20230614,Termo,1,1,1,0,0,0,1,0,0,"É a relevação da falta, de forma que o servidor não sofra qualquer desconto pelo dia não trabalh...","Faltas, Abono das faltas e Abono de ausências.",Inassiduidade habitual e Afastamento de pessoal.,Usado para servidores regidos pela CLT (art. 473).,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1
4,"O termo é ""Abono de permanência em serviço"".\nAbono de permanência em serviço tem definição: ""O ...",5,Abono de permanência em serviço,20230614,Termo,0,1,1,0,0,0,1,0,0,O abono de permanência será pago ao servidor que tiver preenchido as exigências para aposentador...,"Abono de permanência, Adicional de permanência e Abono permanência.","Aposentadoria voluntária, Contribuição previdenciária e Regime Próprio de Previdência Social.",NaN,NaN,NaN,NaN,NaN,NaN,0,7,0,4,11


In [26]:
# Verificar as colunas com valores NaN
columns_with_nan = df_doc.columns[df_doc.isna().any()].tolist()

print(columns_with_nan)

['TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION', 'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION']


In [27]:
df_doc = df_doc.fillna("")

In [28]:
# Converter as colunas para o tipo string
df_doc[columns_with_nan] = df_doc[columns_with_nan].astype(str)

# Expand docs

## Create reference to index (juris_tcu_index)

In [29]:
INDEX_NAME = 'indir_juris_tcu_index'

In [30]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=5: <Document: id=5, content='O termo é "Abono de permanência em serviço".
Abono de permanência em serviço tem definição: "O abono...'>


## Create pipeline

In [31]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [32]:
pipes = []

In [33]:
ranker_type = 'PTT5_INDIR_400' #'PTT5_BASE' #'PTT5_INDIR_83' # 'MT5_3B' # 'MT5_TRAINED_LIM50_800' # 'MINILM_TRAINED_19000'4

In [34]:
FILTER_TERMO = {'class':['Termo']}

In [35]:
FILTER_TERMO, ranker_type

({'class': ['Termo']}, 'PTT5_INDIR_400')

In [36]:
%%time
pipe_join_bm25_sts_reranker = util_pipeline.return_pipeline_join_bm25_sts_reranker(index, ranker_type, parm_limit_query_size=350)

Loading PTT5_INDIR_400 with limit_query_size=350
CPU times: user 11.6 s, sys: 1.82 s, total: 13.4 s
Wall time: 7.14 s


In [37]:
pipes.append({'PIPE_NAME': 'pipe_join_bm25_sts_reranker',
              'PIPE_OBJECT': pipe_join_bm25_sts_reranker,
              'RETRIEVER_TYPE': 'join_sts_bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [42]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 150}, "StsRetriever": {"top_k": 150}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 20)

Parâmetros usados: {'Bm25Retriever': {'top_k': 150}, 'StsRetriever': {'top_k': 150}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Transferência documental', '6139', -0.005041031166911125]
1 : ['Armazenamento de dados', '1101009', -0.01332791242748499]
2 : ['Vazamento de dados', '1110487', -0.017766617238521576]
3 : ['Determinação', '15266', -0.02106134407222271]
4 : ['Gestor', '716', -0.061831917613744736]
5 : ['Gestão documental', '6136', -0.0694652572274208]
6 : ['Culpa in vigilando', '360', -0.23385845124721527]
7 : ['Competência do TCU', '5095', -0.25020989775657654]
8 : ['Gestor público', '5307', -0.2594102621078491]
9 : ['Conduta omis

In [43]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 20)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Armazenamento de dados', '1101009', -0.01332791242748499]
1 : ['Vazamento de dados', '1110487', -0.017766617238521576]
2 : ['Determinação', '15266', -0.02104850485920906]
3 : ['Gestor', '716', -0.061831917613744736]
4 : ['Gestão documental', '6136', -0.069437675178051]
5 : ['Culpa in vigilando', '360', -0.23364831507205963]
6 : ['Competência do TCU', '5095', -0.25020989775657654]
7 : ['Conduta omissiva', '15275', -0.2904728055000305]
8 : ['Arquivamento', '108', -0.30233100056648254]
9 : ['Controle interno', '339'

In [40]:
raise Exception('Stop here!')

Exception: Stop here!

## Create expansion

In [44]:
from unidecode import unidecode
import string
import re


In [45]:
def set_normalize_text(text):
    # Remover espaços duplicados e espaços extras à direita e à esquerda
    text = re.sub(r'\s+', ' ', text.lower()).strip()
    # Remover acentos e caracteres especiais
    text = unidecode(text)
    # Separar as palavras em substrings
    words = text.split()
    # Retornar um conjunto de palavras normalizadas
    return set(words)

In [46]:
def test_similar_or_subtext(text1, text2):
    normalized_text1 = set_normalize_text(text1)
    normalized_text2 = set_normalize_text(text2)
    return normalized_text1 == normalized_text2 or normalized_text1.issubset(normalized_text2)


In [47]:
set_normalize_text('olá comO   vai voc? está bem.')

{'bem.', 'como', 'esta', 'ola', 'vai', 'voc?'}

In [48]:
test_similar_or_subtext("Licitação de técnica e preço","técnica e preço")

False

In [107]:

def expand_query(parm_pipe, parm_df_query, num_expansion):
    if num_expansion > 30:
        raise Exception('Review limit in code for num_expansion of 30!')
    count_expansion = [inteiro + 1 for inteiro in range(num_expansion)]
    expanded_rows = []
    for _, row in tqdm(parm_df_query.iterrows(), total =parm_df_query.shape[0]):
        query_id = row['ID']
        doc_return = parm_pipe.run(query=row['TEXT'], params={"Bm25Retriever": {"top_k": 150}, "StsRetriever": {"top_k": 150}, "filters": FILTER_TERMO}) # , "Ranker": {"top_k": num_expansion*4}

        termos = []
        text_formed = row['TEXT']
        for docto in doc_return['documents'][:num_expansion * 3]:
            if not test_similar_or_subtext(docto.meta['name'], text_formed):
                termos.append(docto.meta['id'])
            if len(termos) == num_expansion:
                break
        
        for count in count_expansion:
            if count <= len(termos):
                doc_id = termos[count-1]
                expanded_rows.append({'QUERY_ID': query_id, 'DOC_ID': doc_id, 'ORDER': count})
    
    df_expanded = pd.DataFrame(expanded_rows)
    return df_expanded


In [108]:
df_query = pd.read_csv(PATH_QUERY)

In [109]:
df_query.head(2)

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE


%%time
df_expanded = expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query.sample(frac=0.0001, random_state=123).reset_index(drop=True), num_expansion=5)

In [112]:
%%time
df_expanded = expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query, num_expansion=5)

  0%|          | 30/16045 [01:10<10:30:42,  2.36s/it]

In [ ]:
EXPANSOR_CRITERIA = 'join_300_ptt5_indir_400' 

In [ ]:
PATH_DOC_EXPANDED =  f'../data/juris_tcu_expanded_doc/doc_5_expansion_{EXPANSOR_CRITERIA}.csv' 

In [ ]:
df_expanded.to_csv(PATH_DOC_EXPANDED, index=False)

In [ ]:
df_expanded.head()

,QUERY_ID,DOC_ID,ORDER
0,31083,1196,1
1,31083,932,2
2,31083,1200,3
3,31083,1289,4
4,31083,1496,5
5,36806,1104186,1
6,36806,16124,2
7,36806,338,3
8,36806,9,4
9,36806,1102013,5


# Generate concatenations

Concatenate terms (by qrel) and synonyms (by doc)

In [72]:
df_expanded.head()

,QUERY_ID,DOC_ID
0,31083,1196
1,31083,932
2,31083,1200
3,31083,1289
4,31083,1496


In [ ]:
df = df_expanded.merge(df_doc[['ID', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM']], how='inner', left_on='DOC_ID', right_on='ID')

In [74]:
df.shape

(10, 8)

In [75]:
df

,QUERY_ID,DOC_ID,ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM
0,31083,1196,1196,Preço global,,"Licitação de menor preço, Método de limitação do preço global e Orçamento detalhado.",0,1
1,31083,932,932,Licitação,"Certame licitatório, Procedimento licitatório, Procedimentos licitatórios, Licitação pública, Li...","Sobrepreço, Superfaturamento, Margem de preferência, Equipe de apoio, Princípio da padronização,...",1,1
2,31083,1200,1200,Preço unitário,,"Licitação de menor preço, Método de limitação de preços unitários ajustados e Orçamento detalhado.",0,1
3,31083,1289,1289,Proposta,Proposta (licitação).,"Pregão, Licitação de melhor técnica, Licitação de maior lance ou oferta, Licitação de menor preç...",1,1
4,31083,1496,1496,Superfaturamento,"Preço superfaturado, Superfaturamento por reajustamento irregular de preço, Superfaturamento de...","Sobrepreço, Faturamento, Licitação e Preço.",1,1
5,36806,1104186,1104186,Procedimento disciplinar,,Pena disciplinar e Infração disciplinar.,0,1
6,36806,16124,16124,(Controle externo),,,0,0
7,36806,338,338,Controle externo,,"Ação de controle externo, Órgão de controle externo, Administração estadual, Lei Orgânica do TCU...",0,1
8,36806,9,9,Ato disciplinar,Ação disciplinar.,,1,0
9,36806,1102013,1102013,Órgão de controle externo,"Organizações de controle externo, Entidade de controle externo, Unidade de controle externo, Uni...","Órgão de controle interno, Controle externo, Processo de controle externo, Processo de auditoria...",1,1


In [76]:
del df['ID']

In [77]:
df.head()

,QUERY_ID,DOC_ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM
0,31083,1196,Preço global,,"Licitação de menor preço, Método de limitação do preço global e Orçamento detalhado.",0,1
1,31083,932,Licitação,"Certame licitatório, Procedimento licitatório, Procedimentos licitatórios, Licitação pública, Li...","Sobrepreço, Superfaturamento, Margem de preferência, Equipe de apoio, Princípio da padronização,...",1,1
2,31083,1200,Preço unitário,,"Licitação de menor preço, Método de limitação de preços unitários ajustados e Orçamento detalhado.",0,1
3,31083,1289,Proposta,Proposta (licitação).,"Pregão, Licitação de melhor técnica, Licitação de maior lance ou oferta, Licitação de menor preç...",1,1
4,31083,1496,Superfaturamento,"Preço superfaturado, Superfaturamento por reajustamento irregular de preço, Superfaturamento de...","Sobrepreço, Faturamento, Licitação e Preço.",1,1


In [78]:
df['NAME_SYNONYM'] = df.apply(lambda row: row['NAME'] if row['TEXT_SYNONYM'] == '' else row['NAME'] + ' - ' + row['TEXT_SYNONYM'], axis=1)


In [79]:
df['NAME_RELATED_TERM'] = df.apply(lambda row: row['NAME'] if row['TEXT_RELATED_TERM'] == '' else row['NAME'] + ' -- ' + row['TEXT_RELATED_TERM'], axis=1)


In [80]:
df['NAME_SYNONYM_RELATED_TERM'] = df.apply(lambda row: row['NAME_SYNONYM'] if row['TEXT_RELATED_TERM'] == '' else row['NAME_SYNONYM'] + ' -- ' + row['TEXT_RELATED_TERM'], axis=1)


In [81]:
df[(df['HAS_SYNONYM']==1) & (df['HAS_RELATED_TERM']==0)].head(10)

,QUERY_ID,DOC_ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
8,36806,9,Ato disciplinar,Ação disciplinar.,,1,0,Ato disciplinar - Ação disciplinar.,Ato disciplinar,Ato disciplinar - Ação disciplinar.


In [82]:
df = df_query.merge(df, how='inner', left_on='ID', right_on='QUERY_ID')

In [83]:
df.columns

Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME', 'QUERY_ID', 'DOC_ID', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM', 'NAME_RELATED_TERM',
       'NAME_SYNONYM_RELATED_TERM'],
      dtype='object')

In [84]:
df.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,QUERY_ID,DOC_ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
0,31083,Não há que se falar em sobrepreço em determinados serviços constantes das planilhas de preços un...,NaN,NaN,Contrato administrativo,331,RELATÓRIO DE LEVANTAMENTO,Acórdão 2046/2008 - Plenário,2008-09-17,RELATOR,UBIRATAN AGUIAR,31083,1196,Preço global,,"Licitação de menor preço, Método de limitação do preço global e Orçamento detalhado.",0,1,Preço global,"Preço global -- Licitação de menor preço, Método de limitação do preço global e Orçamento detalh...","Preço global -- Licitação de menor preço, Método de limitação do preço global e Orçamento detalh..."
1,31083,Não há que se falar em sobrepreço em determinados serviços constantes das planilhas de preços un...,NaN,NaN,Contrato administrativo,331,RELATÓRIO DE LEVANTAMENTO,Acórdão 2046/2008 - Plenário,2008-09-17,RELATOR,UBIRATAN AGUIAR,31083,932,Licitação,"Certame licitatório, Procedimento licitatório, Procedimentos licitatórios, Licitação pública, Li...","Sobrepreço, Superfaturamento, Margem de preferência, Equipe de apoio, Princípio da padronização,...",1,1,"Licitação - Certame licitatório, Procedimento licitatório, Procedimentos licitatórios, Licitação...","Licitação -- Sobrepreço, Superfaturamento, Margem de preferência, Equipe de apoio, Princípio da ...","Licitação - Certame licitatório, Procedimento licitatório, Procedimentos licitatórios, Licitação..."
2,31083,Não há que se falar em sobrepreço em determinados serviços constantes das planilhas de preços un...,NaN,NaN,Contrato administrativo,331,RELATÓRIO DE LEVANTAMENTO,Acórdão 2046/2008 - Plenário,2008-09-17,RELATOR,UBIRATAN AGUIAR,31083,1200,Preço unitário,,"Licitação de menor preço, Método de limitação de preços unitários ajustados e Orçamento detalhado.",0,1,Preço unitário,"Preço unitário -- Licitação de menor preço, Método de limitação de preços unitários ajustados e ...","Preço unitário -- Licitação de menor preço, Método de limitação de preços unitários ajustados e ..."
3,31083,Não há que se falar em sobrepreço em determinados serviços constantes das planilhas de preços un...,NaN,NaN,Contrato administrativo,331,RELATÓRIO DE LEVANTAMENTO,Acórdão 2046/2008 - Plenário,2008-09-17,RELATOR,UBIRATAN AGUIAR,31083,1289,Proposta,Proposta (licitação).,"Pregão, Licitação de melhor técnica, Licitação de maior lance ou oferta, Licitação de menor preç...",1,1,Proposta - Proposta (licitação).,"Proposta -- Pregão, Licitação de melhor técnica, Licitação de maior lance ou oferta, Licitação d...","Proposta - Proposta (licitação). -- Pregão, Licitação de melhor técnica, Licitação de maior lanc..."
4,31083,Não há que se falar em sobrepreço em determinados serviços constantes das planilhas de preços un...,NaN,NaN,Contrato administrativo,331,RELATÓRIO DE LEVANTAMENTO,Acórdão 2046/2008 - Plenário,2008-09-17,RELATOR,UBIRATAN AGUIAR,31083,1496,Superfaturamento,"Preço superfaturado, Superfaturamento por reajustamento irregular de preço, Superfaturamento de...","Sobrepreço, Faturamento, Licitação e Preço.",1,1,"Superfaturamento - Preço superfaturado, Superfaturamento por reajustamento irregular de preço, ...","Superfaturamento -- Sobrepreço, Faturamento, Licitação e Preço.","Superfaturamento - Preço superfaturado, Superfaturamento por reajustamento irregular de preço, ..."


In [85]:
df.shape

(10, 21)

In [ ]:
GERAR ATÉ 1, 3 e 5

In [ ]:
%%time
data_index = {'term' : [],
            'synonym': [],
            'related_term': [],
            'synonym_related_term': [],}
# Ordenar o DataFrame por 'QUERY_ID' em ordem ascendente
df_sorted = df.sort_values('QUERY_ID')
# Iterar sobre os grupos
for cnt, (query_id, group) in tqdm(enumerate(df_sorted.groupby('QUERY_ID'))):
    # print(f"Query ID: {query_id}")
    # print(group)
    group_synonym = ''
    group_term = ''
    group_related_term = ''
    group_synonym_related_term = ''

    # Iterar sobre os registros do grupo
    for cnt, (_, row_doc) in enumerate(group.iterrows()):
        if cnt==0: # salva copia dos dados da query
            row_query = row_doc.copy()

        doc_id = row_doc['DOC_ID']
        # Faça o que precisa com cada registro (doc_id) para o query_id atual
        
        # Exemplo: Imprimir o doc_id
        # print(f"Doc ID: {doc_id}")

        group_term = group_term + ', ' + row_doc['NAME'] if group_term != '' else row_doc['NAME']
        if row_doc['HAS_SYNONYM']==1:
            group_synonym = group_synonym + ' ' + row_doc['NAME_SYNONYM'] if group_synonym != '' else row_doc['NAME_SYNONYM']
        if row_doc['HAS_RELATED_TERM']==1:
            group_related_term = group_related_term + ' ' + row_doc['NAME_RELATED_TERM'] if group_related_term != '' else row_doc['NAME_RELATED_TERM']
        if (row_doc['HAS_RELATED_TERM']==1) or (row_doc['HAS_SYNONYM']==1):
            group_synonym_related_term = group_synonym_related_term + ' ' + row_doc['NAME_SYNONYM_RELATED_TERM'] if group_synonym_related_term != '' else row_doc['NAME_SYNONYM_RELATED_TERM']
    # Adicione aqui qualquer lógica adicional que você precise executar para cada query_id

    row_json_term= {
        'id': row_query['ID'],
        'content': row_query['TEXT'],
        'meta': {},
    }               
    for column_name in df.columns:
        if column_name not in ['TEXT', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME', 'TEXT_SYNONYM',
                               'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM',
                               'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM']:  # columns of doc
            row_json_term['meta'][column_name.lower()] = row_query[column_name]        
    
    row_json_synonym = row_json_term.copy()
    row_json_related_term = row_json_term.copy()
    row_json_synonym_related_term = row_json_term.copy()

    row_json_term['content'] += '\n' + group_term + '.'
    if len(group_synonym) > 0:
        row_json_synonym['content'] += '\n' + group_synonym
    if len(group_related_term) > 0:
        row_json_related_term['content'] += '\n' + group_related_term
    if len(group_synonym_related_term) > 0:
        row_json_synonym_related_term['content'] += '\n' + group_synonym_related_term


    data_index['term'].append(row_json_term)
    data_index['synonym'].append(row_json_synonym)
    data_index['related_term'].append(row_json_related_term)
    data_index['synonym_related_term'].append(row_json_synonym_related_term)


16045it [00:07, 2217.00it/s]

CPU times: user 7.25 s, sys: 60.3 ms, total: 7.31 s
Wall time: 7.27 s


In [ ]:
len(data_index['term'])

16045

In [ ]:
list_type_index = ['term', 'synonym', 'related_term', 'synonym_related_term' ]

In [ ]:
ind_test=1
for index_type in list_type_index:
    print(index_type, data_index[index_type][ind_test]['content'], '\n')


term SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão, Doença, Pessoal, Viúvo, Pensão especial. 

synonym SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão - Ocupante de cargo em comissão, Cargo de direção, chefia e assessoramento, Cargo comissionado, Cargo de confiança, Exercente de cargo em comissão e Cargo de direção, chefia ou assessoramento. Doença - Enfermidade e Moléstia. Viúvo - Viúva, Cônjuge sobrevivente, Cônjuge supérstite e Consorte supérstite. Pensão especial - Pensão especial da Lei 3738/60, Pensão especial da Lei 6782/80, Pensão da lei 3.738/60 e Pensão es

# Index creation

In [ ]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu', 'uuid': 'RD_aAWppQH6vjNxKUdL_0w', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '334.9mb', 'pri.store.size': '334.9mb'}

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'fGjzBrQzRUWTZ6KlnbfLqw', 'pri': '1', 'rep': '1', 'docs.count': '13255', 'docs.deleted': '2755', 'store.size': '281.7mb', 'pri.store.size': '281.7mb'}



In [ ]:
INDEX_NAME = 'indir_juris_tcu'

In [ ]:
# util_elastic.delete_index('indir_juris_tcu')

In [ ]:
%%time
dict_index = {}
for index_type in list_type_index:
    print(f"\n********Creating index {INDEX_NAME+'_'+index_type}")
    dict_index[index_type] = util_elastic.create_index(parm_index_name=INDEX_NAME+'_'+index_type,
                                                       parm_data_carga_json=data_index[index_type],
                                                       parm_embedding_dim=1024)

Creating index indir_juris_tcu_term

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index indir_juris_tcu_synonym

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index indir_juris_tcu_related_term

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index in

In [ ]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [ ]:
%%time
for index_type in list_type_index:
    print(f"\n********Updating embeddings of index {INDEX_NAME+'_'+index_type}")
    util_elastic.update_index_embedding_sts(parm_index=dict_index[index_type], parm_path_model=nome_caminho_modelo)


********Updating embeddings of index indir_juris_tcu_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_related_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_related_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]

CPU times: user 10min 41s, sys: 3.99 s, total: 10min 45s
Wall time: 13min 49s
